In [1]:
import sys
sys.path.append("../")

In [5]:
import random

from qwrapper.obs import PauliObservable
from qwrapper.hamiltonian import HeisenbergModel, compute_ground_state
from qwrapper.optimizer import AdamOptimizer,AdamOptimizerGPU, UnitLRScheduler, PrintMonitor, FileMonitor
from qswift.initializer import XBasisInitializer
from gqe.simple_model.model import SimpleModel, Ansatz
from gqe.energy_estimator.iid import IIDEstimator

In [6]:
# The file path, where the model is saved
OUTPUT_FILENAME = '../saved_models/model_three.json'

In [7]:
N = 8000
n_sample = 1000
lam = 15
nqubit = 3

# Hamiltonian and its theoretical ground state energy

In [8]:
hamiltonian = HeisenbergModel(nqubit)
print(compute_ground_state(hamiltonian))

-3.0000000000000004


# Initialize the operator pool
We define the set of operators $\{O_j\}$. Then, each gate has the form $e^{i\theta_j O_j}$. The parameters are initialized by the gaussian distribution.

In [9]:
ansatz = Ansatz([random.gauss(0, 0.5) for _ in range(18)],
                [
                    PauliObservable("XII"), PauliObservable("YII"), PauliObservable("ZII"),
                    PauliObservable("IXI"), PauliObservable("IYI"), PauliObservable("IZI"),
                    PauliObservable("IIX"), PauliObservable("IIY"), PauliObservable("IIZ"),
                    PauliObservable("XXI"), PauliObservable("YYI"), PauliObservable("ZZI"),
                    PauliObservable("IXX"), PauliObservable("IYY"), PauliObservable("IZZ"),
                    PauliObservable("XIX"), PauliObservable("YIY"), PauliObservable("ZIZ"),
                ], nqubit=nqubit)

# Initialize Energy Estimator
- The module calculates the expectation value of the Hamiltonian and its gradient for each parameter, assuming that the gate at each position is generated from an i.i.d. distribution..
- XBasisInitializer determines the initial quantum state before applying the generated quantum gates.

In [11]:
estimator = IIDEstimator(hamiltonian,
                         XBasisInitializer(),
                         N, K=0, tool='qulacs-gpu', n_sample=n_sample, n_grad_sample=1)

# Perform simple Model optimization
The model generate the sequence of gates, calculate the gradient, and update the parameters.

In [12]:
model = SimpleModel(estimator, ansatz, N, lam, tool="qulacs-gpu")
monitors = [PrintMonitor(), FileMonitor('output/energy.txt')]
model.run(AdamOptimizerGPU(maxiter=100, scheduler=UnitLRScheduler(0.01), monitors=monitors,device="cuda"))


0 0.8849647183622495
1 0.7440626743392483
2 0.6010760924468654
3 0.4569139188198652
4 0.3151411102944675
5 0.17965005373359075
6 0.03854695840076067
7 -0.10196219441771609
8 -0.23730036120844417
9 -0.3709895454561515
10 -0.5013484279310048
11 -0.624328757454928
12 -0.7447007593362963
13 -0.8561682673311874
14 -0.9638746265435083
15 -1.0612308198145373
16 -1.15556983116814
17 -1.2446230982382085
18 -1.3267756587061372
19 -1.4016791193758524
20 -1.4693122614874738
21 -1.5333531634482884
22 -1.5968558470015393
23 -1.6571450396072358
24 -1.7137738552645718
25 -1.7680806074916833
26 -1.8202254001224094
27 -1.8728126515588772
28 -1.9225040094724446
29 -1.9717575343551093
30 -2.0193070400245166
31 -2.067155644413165
32 -2.113797118733145
33 -2.1590166138995626
34 -2.204999247206332
35 -2.250318245816521
36 -2.29583108911939
37 -2.341035568757426
38 -2.386254114481071
39 -2.4293275088744957
40 -2.4694399391315924
41 -2.5083479006164024
42 -2.5462998108116066
43 -2.583498725783288
44 -2.6186480

In [ ]:
for m in monitors:
    m.finalize()
with open(OUTPUT_FILENAME, 'w') as f:
    f.write(model.ansatz.toJSON())